In [89]:
import os
import praw
import csv
import time
import random
import requests
from praw.models import Comment
from prawcore.exceptions import ResponseException

In [91]:
# ✅ Initialize Reddit API with async warning suppressed
reddit = praw.Reddit(
    client_id="QE1fYv3WZI2oV8TJhln90w",
    client_secret="2-AVnhWkZo-3Bq9EH57kL7l9XVj-Og",
    user_agent="mental health post scraper by Dangerous_Emu_7611"
)

In [107]:
# 🔍 Search terms
search_terms = ['mental health', 'depression', 'anxiety', 'stress']
posts = []

In [109]:
# --- Try searching r/Malaysia for each keyword ---
subreddit = reddit.subreddit('Malaysia')

for term in search_terms:
    print(f"Searching for '{term}' in r/Malaysia...")
    try:
        for submission in subreddit.search(term, limit=50):
            posts.append(submission)
            time.sleep(random.uniform(2, 4))  # Be respectful to API
    except Exception as e:
        print(f"⚠️ Could not search for '{term}'. Error: {e}")
        print("Falling back to fetching new posts...")

        for submission in subreddit.new(limit=50):
            posts.append(submission)
            time.sleep(random.uniform(2, 4))

# --- Save to CSV ---
with open('mentalhealth.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'post_id', 'title', 'selftext', 'post_score', 'upvote_ratio',
        'num_comments', 'created_utc',
        'comment_id', 'comment_body', 'comment_score',
        'comment_awards', 'comment_created_utc'
    ])

    for submission in posts:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            writer.writerow([
                submission.id,
                submission.title,
                submission.selftext.replace('\n', ' ').replace('\r', ''),
                submission.score,
                submission.upvote_ratio,
                submission.num_comments,
                submission.created_utc,
                comment.id,
                comment.body.replace('\n', ' ').replace('\r', '').strip(),
                comment.score,
                comment.total_awards_received,
                comment.created_utc
            ])

print("✅ Done! Data saved to mentalhealth.csv")

Searching for 'mental health' in r/Malaysia...
Searching for 'depression' in r/Malaysia...
Searching for 'anxiety' in r/Malaysia...
Searching for 'stress' in r/Malaysia...
✅ Done! Data saved to mentalhealth.csv


In [111]:
import pandas as pd
mentalhealth = pd.read_csv('mentalhealth.csv')
mentalhealth

,post_id,title,selftext,post_score,upvote_ratio,num_comments,created_utc,comment_id,comment_body,comment_score,comment_awards,comment_created_utc
0,1h704et,Malaysian psychiatrist with ‘promising career’...,NaN,137,0.94,46,1.733371e+09,m0j5twr,Raped. He raped a minor entrusted under his ca...,27,0,1.733404e+09
1,1h704et,Malaysian psychiatrist with ‘promising career’...,NaN,137,0.94,46,1.733371e+09,m0hiscs,Apparently this dude is bro of Dr halina wife ...,74,0,1.733371e+09
2,1h704et,Malaysian psychiatrist with ‘promising career’...,NaN,137,0.94,46,1.733371e+09,m0hs8q8,"Like my mom always asked, 'Anak siapa ni?'",18,0,1.733375e+09
3,1h704et,Malaysian psychiatrist with ‘promising career’...,NaN,137,0.94,46,1.733371e+09,m0hpmsh,> She reportedly said the married Amirul Arif ...,28,0,1.733374e+09
4,1h704et,Malaysian psychiatrist with ‘promising career’...,NaN,137,0.94,46,1.733371e+09,m0hjkkv,nerakazens are doing their job at x. hehehe ...,12,0,1.733371e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
13718,oj9ooa,Hidup kena happy.. Rehat jap.. Hilangkan stress 😄,NaN,183,0.94,15,1.626157e+09,h51smfn,Hahahaa so funny eh? What about the pakages? T...,-7,0,1.626192e+09
13719,oj9ooa,Hidup kena happy.. Rehat jap.. Hilangkan stress 😄,NaN,183,0.94,15,1.626157e+09,h50tvo7,No wonder my package missing!,15,0,1.626173e+09
13720,oj9ooa,Hidup kena happy.. Rehat jap.. Hilangkan stress 😄,NaN,183,0.94,15,1.626157e+09,h51h5pi,Haha just allowed it bro,1,0,1.626187e+09
13721,oj9ooa,Hidup kena happy.. Rehat jap.. Hilangkan stress 😄,NaN,183,0.94,15,1.626157e+09,h51fswb,Biar lambat asalkan selamat,4,0,1.626186e+09
